<a href="https://colab.research.google.com/github/mading225/GROUP-6-PROJECT-/blob/main/Movie_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Understanding

## Business Problem

The company is planning to enter the entertainment industry by launching a new movie studio. While the company has experience in other business areas, it currently lacks expertise in movie production and distribution.

Creating movies requires significant upfront investment, and poor decisions around genre, audience targeting, or production strategy can lead to major financial losses. To reduce this risk, the company needs data-driven guidance on what types of films are most likely to succeed in today’s market.

## Business Objectives

The objective of this project is to **analyze historical movie data** to identify patterns associated with **financial success and positive audience reception.**

Using box office performance and audience ratings, this analysis aims to answer the following questions:

 - What types of movies generate the highest box office revenue?

 - Which movie characteristics are associated with strong audience approval?

 - What trends can guide the studio’s initial movie production strategy?

## Key Business Questions

To support leadership decision-making, this project focuses on three core questions:

 - **Which movie genres consistently perform best at the box office?**

 - **How does audience reception (ratings and engagement) relate to financial success?**

 - **What movie characteristics (such as runtime or release trends) are most associated with successful films?**

# Data Understanding

To explore what types of movies perform best both financially and critically, this project uses three datasets sourced from widely recognized movie industry platforms. Each dataset provides a different perspective on movie performance, allowing for a more well-rounded analysis.

In [1]:
# Importing the necessary libraries
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

## IMDB Dataset

The IMDB data is stored in a SQLite database. This database contains multiple tables, but this analysis focuses on:

`movie_basics`

`movie_ratings`

These tables provide movie characteristics and audience ratings.

In [2]:
# Unzipping the zipped imdb dataset
!unzip im.db.zip -d zippedData


Archive:  im.db.zip
  inflating: zippedData/im.db        


In [3]:
# Connect to Sqlite3
conn = sqlite3.connect('zippedData/im.db')

# Load relevant tables
movie_basics = pd.read_sql("""
SELECT * FROM movie_basics;
""", conn)

movie_ratings = pd.read_sql("""
SELECT * FROM movie_ratings;
""", conn)



In [4]:
# Previewing the movie_basics table
movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [5]:
# Previewing the movie_ratings table
movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


## Box Office Mojo Dataset (*bom.movie_gross.csv.gz*)


This dataset contains box office revenue data for movies released domestically and internationally.

In [8]:
# Loading and Previewing the gross dataset
gross = pd.read_csv('bom.movie_gross.csv.gz')
gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


The Box Office Mojo dataset was provided as a gzip-compressed CSV file, which was read directly using pandas without the need for manual extraction.

## The Movie Database (*tmdb.movies.csv.gz*)

The TMDB dataset provides detailed information about movies, including popularity metrics and audience ratings, which complement the IMDB and Box Office Mojo datasets.

In [11]:
# Loading and previewing the tmdb dataset
movies = pd.read_csv('tmdb.movies.csv.gz')
movies.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


# Data Preparation

Since we have loaded in the datasets and previewed them, we are going to prepare them for modelling and visualisations.

We will prepare our datasets by; cleaning, removing duplicates and finally merging all our three datasets into one.

## IMDB Datasets

The IMDB holds two datasets, `movie_basics` and `movie_ratings`. These two datasets will have to be cleaned individually then once we are done with cleaning, we will merge both the datasets into one.


### Movie_basics Dataset

In [12]:
# Checking for any null values
movie_basics.isnull().sum()

,0
movie_id,0
primary_title,0
original_title,21
start_year,0
runtime_minutes,31739
genres,5408


In [13]:
# Removing null values in the primary title column
movie_basics.dropna(subset=['primary_title'], inplace=True)
movie_basics['primary_title'].isnull().sum()

np.int64(0)

In [14]:
# Removing null values from the runtime column
movie_basics.dropna(subset=['runtime_minutes'], inplace=True)
movie_basics['runtime_minutes'].isnull().sum()

np.int64(0)

In [15]:
# Removing null values from the genre column
movie_basics.dropna(subset=['genres'], inplace=True)
movie_basics['genres'].isnull().sum()

np.int64(0)

In [16]:
# Dropping the original title column
movie_basics = movie_basics.drop(columns=['original_title'])
movie_basics

,movie_id,primary_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy"
5,tt0111414,A Thin Life,2018,75.0,Comedy
...,...,...,...,...,...
146134,tt9916160,Drømmeland,2019,72.0,Documentary
146135,tt9916170,The Rehearsal,2019,51.0,Drama
146136,tt9916186,Illenau - die Geschichte einer ehemaligen Heil...,2017,84.0,Documentary
146137,tt9916190,Safeguard,2019,90.0,"Drama,Thriller"


In [17]:
movie_basics.duplicated().sum()

np.int64(0)

We have managed to remove all null values from the `movie_basics` dataset.

We also dropped the *original_title* column because it holds no value to us since it has the original names of certain movie titles which will only hold the dataset back during merging.

The dataset held no duplicates.

### Movie_ratings Dataset

In [18]:
movie_ratings.isnull().sum()

,0
movie_id,0
averagerating,0
numvotes,0


Since the `movie_ratings` dataset holds no null values, we proceed over to merging.

### Merging `movie_basics` and `movie_ratings`

In [21]:
# Step 1: Merge on movie_id
merged_imdb = pd.merge(
    movie_basics,
    movie_ratings,
    left_on= 'movie_id', # movie_basics
    right_on= 'movie_id', # movie_ratings
    how='inner'
)

# Step 3: Arranging the columns
merged_imdb = merged_imdb[[
    'movie_id',
    'primary_title',
    'start_year',
    'runtime_minutes',
    'genres',
    'averagerating'
]]


print("Merged shape:", merged_imdb.shape)
merged_imdb.head()

Merged shape: (65720, 6)


,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9
3,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",6.5
4,tt0137204,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",8.1


We have merged the two datasets by using the common column being *movie_id* which resulted in rows with matching *movie_id* to be chosen and resulted in the `merged_imdb` dataset

## Box Office Mojo Dataset

In [22]:
gross.isnull().sum()

,0
title,0
studio,5
domestic_gross,28
foreign_gross,1350
year,0


In [23]:
# Dropping the null values in the studio column
gross.dropna(subset=['studio'], inplace=True)
gross['studio'].isnull().sum()

np.int64(0)

In [24]:
# Dropping the null values in the domestic gross column
gross.dropna(subset=['domestic_gross'], inplace=True)
gross['domestic_gross'].isnull().sum()

np.int64(0)

In [25]:
# Dropping the null values in the foreign gross column
gross.dropna(subset=['foreign_gross'], inplace=True)
gross['foreign_gross'].isnull().sum()

np.int64(0)

In [29]:
# Combining domestic and foreign gross column into total_gross

# Step 1: Convert to float first to handle decimals
gross['domestic_gross'] = pd.to_numeric(gross['domestic_gross'])
gross['foreign_gross'] = pd.to_numeric(gross['foreign_gross'])

# Step 2: Convert to integer
gross['domestic_gross'] = gross['domestic_gross'].astype(int)
gross['foreign_gross'] = gross['foreign_gross'].astype(int)

# Step 3: Create total_gross
gross['total_gross'] = gross['domestic_gross'] + gross['foreign_gross']

# Step 4: Removing domestic and foreign gross columns
gross = gross.drop(columns=['domestic_gross', 'foreign_gross'])
gross

,title,studio,year,total_gross
0,Toy Story 3,BV,2010,1067000000
1,Alice in Wonderland (2010),BV,2010,1025500000
2,Harry Potter and the Deathly Hallows Part 1,WB,2010,960300000
3,Inception,WB,2010,828300000
4,Shrek Forever After,P/DW,2010,752600000
...,...,...,...,...
3275,I Still See You,LGF,2018,1501400
3286,The Catcher Was a Spy,IFC,2018,954000
3309,Time Freak,Grindstone,2018,266000
3342,Reign of Judges: Title of Liberty - Concept Short,Darin Southa,2018,98400


We have managed to remove all null values from the `gross` dataset.

We merged and dropped the domestic and foreign gross columns to form a *total_gross* column.